## Fashion-MNIST-based simulated production line prediction
<font color=#FF0000>Description, TO BE DONE!!</font>

### Packages import

In [61]:
# necessary packages
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random

# additional certain short functions
from torch import is_tensor
from matplotlib.pyplot import pause
from random import randint
from copy import deepcopy
from math import floor, ceil
from torch import stack
from torch import cat

# import custom functions
from data_generate import *
from data_io import *
from CNN import *
from FCN import *

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [62]:
# apply CUDA
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


### Custom data loading

In [63]:
# Customize dataset
class ProductLineDataset(Dataset):
    def __init__(self, image: torch.Tensor, gt: torch.Tensor):
        self.x = image.reshape(len(image), 1, 28*3, 28*4).type(torch.float32)
        self.y = gt.type(torch.float32)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [92]:
# define file names
CSV_NAME_TRAIN_02346 = 'train_02346.csv'
CSV_NAME_TEST_02346 = 'test_02346.csv'
CSV_NAME_TEST_579 = 'test_579.csv'
CSV_NAME_TEST_18 = 'test_18.csv'
IMAGE_NAME_TRAIN_02346 = 'train_02346'
IMAGE_NAME_TEST_02346 = 'test_02346'
IMAGE_NAME_TEST_579 = 'test_579'
IMAGE_NAME_TEST_18 = 'test_18'

# import data
# 1-D ground truth
gt_train_02346 = gt_import(CSV_NAME_TRAIN_02346)
gt_test_02346 = gt_import(CSV_NAME_TEST_02346)
gt_test_579 = gt_import(CSV_NAME_TEST_579)
gt_test_18 = gt_import(CSV_NAME_TEST_18)

# 2-D ground truth
gt_train_02346 = torch.cat((gt_train_02346.reshape(len(gt_train_02346), 1), torch.zeros(len(gt_train_02346), 1)), dim = 1)
gt_test_02346 = torch.cat((gt_test_02346.reshape(len(gt_test_02346), 1), torch.zeros(len(gt_test_02346), 1)), dim = 1)
gt_test_579 = torch.cat((gt_test_579.reshape(len(gt_test_579), 1), torch.zeros(len(gt_test_579), 1)), dim = 1)
gt_test_18 = torch.cat((gt_test_18.reshape(len(gt_test_18), 1), torch.zeros(len(gt_test_18), 1)), dim = 1)

# 4-D image data
image_train_02346 = image_import(gt_train_02346, IMAGE_NAME_TRAIN_02346)/255.
image_test_02346 = image_import(gt_test_02346, IMAGE_NAME_TEST_02346)/255.
image_test_579 = image_import(gt_test_579, IMAGE_NAME_TEST_579)/255.
image_test_18 = image_import(gt_test_18, IMAGE_NAME_TEST_18)/255.

In [93]:
# print shapes
print(gt_train_02346.shape)
print(image_train_02346.shape)

torch.Size([10415, 2])
torch.Size([10415, 1, 84, 112])


In [95]:
# define dataset
dataset_train_02346 = ProductLineDataset(image_train_02346, gt_train_02346)
dataset_test_02346 = ProductLineDataset(image_test_02346, gt_test_02346)
dataset_test_579 = ProductLineDataset(image_test_579, gt_test_579)
dataset_test_18 = ProductLineDataset(image_test_18, gt_test_18)

# define dataloader
dataloader_train_02346 = DataLoader(dataset_train_02346, batch_size=64, shuffle=True)
dataloader_test_02346 = DataLoader(dataset_test_02346, batch_size=1, shuffle=True)  # batch_size=1 for test
dataloader_test_579 = DataLoader(dataset_test_579, batch_size=1, shuffle=True)
dataloader_test_18 = DataLoader(dataset_test_18, batch_size=1, shuffle=True)

In [96]:
# visualize the shape
for X, y in dataloader_train_02346:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape)
    print("Data type of y: ", y.dtype)
    print(y)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 84, 112])
Shape of y:  torch.Size([64, 2])
Data type of y:  torch.float32
tensor([[4., 0.],
        [5., 0.],
        [3., 0.],
        [3., 0.],
        [3., 0.],
        [6., 0.],
        [3., 0.],
        [3., 0.],
        [5., 0.],
        [7., 0.],
        [3., 0.],
        [3., 0.],
        [7., 0.],
        [7., 0.],
        [3., 0.],
        [5., 0.],
        [5., 0.],
        [6., 0.],
        [3., 0.],
        [5., 0.],
        [3., 0.],
        [5., 0.],
        [5., 0.],
        [5., 0.],
        [7., 0.],
        [4., 0.],
        [6., 0.],
        [4., 0.],
        [3., 0.],
        [4., 0.],
        [3., 0.],
        [3., 0.],
        [3., 0.],
        [4., 0.],
        [5., 0.],
        [3., 0.],
        [5., 0.],
        [7., 0.],
        [3., 0.],
        [7., 0.],
        [4., 0.],
        [4., 0.],
        [4., 0.],
        [4., 0.],
        [4., 0.],
        [4., 0.],
        [4., 0.],
        [6., 0.],
        [6., 0.],

### Define functions

In [141]:
# define functions 
def train(model, dataloader, loss_fn, optimizer):   # put epoch in main better for loss calculation
    # size of dataset
    size = len(dataloader.dataset)
    
    # set model mode
    model.train()
    
    # train batches per epoch
    for batch, (X, y) in enumerate(dataloader):
        # move data to device
        X, y = X.to(device), y.to(device)
        
        score = model(X)
        loss = loss_fn(score, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 40 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(model, dataloader, loss_fn):
    # size of dataset
    size = len(dataloader.dataset)
    
    # number of batches
    num_batches = len(dataloader)
    
    # set model mode
    model.eval()
    
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X,y in dataloader:
            # move data to device
            X, y = X.to(device), y.to(device)
            
            score = model(X)
            test_loss += loss_fn(score, y).item()
            correct += 1 - abs(round(score[0][0].item()) - y[0][0].item()) / y[0][0].item()    # only for batch_size=1
    
    # calculate the average loss and accuracy
    test_loss /= num_batches
    correct /= num_batches
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### Custom model

In [142]:
in_channel = (28*3)*(28*4)
node_1 = 4096
node_2 = 4096
node_3 = 1024
node_4 = 1024
node_5 = 64
node_6 = 64
out_channel = 2

FCN_model = FCN(in_channel, node_1, node_2, node_3, node_4, \
    node_5, node_6, out_channel).to(device=device)
print(FCN_model)

FCN(
  (fc1): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=9408, out_features=4096, bias=True)
    (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Dropout(p=0.5, inplace=False)
  )
  (fc2): Sequential(
    (0): Linear(in_features=4096, out_features=4096, bias=True)
    (1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
  )
  (fc3): Sequential(
    (0): Linear(in_features=4096, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (fc4): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (fc5): Sequ

In [143]:
in_channel = 1
channel_1 = 16
channel_2 = 16
channel_3 = 8
node_1 = 4096
node_2 = 4096
node_3 = 1024
node_4 = 256
out_channel = 2

CNN_model = CNN(in_channel, channel_1, channel_2, channel_3, \
    node_1, node_2, node_3, node_4, out_channel).to(device=device)
print(CNN_model)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1120, out_features=4096, bias=True

### Model setup and train/test

In [144]:
# define parameters
learning_rate = 1e-3

# define lose function
loss_fn = nn.MSELoss()

# define optimizer
optimizer_FCN = torch.optim.Adam(FCN_model.parameters(), lr=learning_rate)
optimizer_CNN = torch.optim.Adam(CNN_model.parameters(), lr=learning_rate)

In [145]:
# set epoch
epoch = 5

# start training
for t in range(epoch):
    print(f"Epoch {t+1}\n-------------------------------")
    train(FCN_model, dataloader_train_02346, loss_fn, optimizer_FCN)
    test(FCN_model, dataloader_test_02346, loss_fn)
print("Done!!")

# model saving
torch.save(FCN_model.state_dict(), "FCN.pth")

Epoch 1
-------------------------------
loss: 12.348166  [    0/10415]
loss: 1.323632  [ 2560/10415]
loss: 0.798921  [ 5120/10415]
loss: 1.025570  [ 7680/10415]
loss: 0.734814  [10240/10415]
Test Error: 
 Accuracy: 80.2%, Avg loss: 0.842968 

Epoch 2
-------------------------------
loss: 0.974456  [    0/10415]
loss: 0.664802  [ 2560/10415]
loss: 0.546709  [ 5120/10415]
loss: 0.793517  [ 7680/10415]
loss: 0.688872  [10240/10415]
Test Error: 
 Accuracy: 82.1%, Avg loss: 0.835731 

Epoch 3
-------------------------------
loss: 0.697943  [    0/10415]
loss: 0.768792  [ 2560/10415]
loss: 0.679389  [ 5120/10415]
loss: 0.853266  [ 7680/10415]
loss: 0.438562  [10240/10415]
Test Error: 
 Accuracy: 80.3%, Avg loss: 1.035536 

Epoch 4
-------------------------------
loss: 0.628119  [    0/10415]
loss: 0.587405  [ 2560/10415]
loss: 0.888324  [ 5120/10415]
loss: 0.647689  [ 7680/10415]
loss: 0.525349  [10240/10415]
Test Error: 
 Accuracy: 78.3%, Avg loss: 1.424564 

Epoch 5
-----------------------

In [146]:
# set epoch
epoch = 5

# start training
for t in range(epoch):
    print(f"Epoch {t+1}\n-------------------------------")
    train(CNN_model, dataloader_train_02346, loss_fn, optimizer_CNN)
    test(CNN_model, dataloader_test_02346, loss_fn)
print("Done!!")

# model saving
torch.save(CNN_model.state_dict(), "CNN.pth")

Epoch 1
-------------------------------
loss: 12.197427  [    0/10415]
loss: 0.753928  [ 2560/10415]
loss: 0.450393  [ 5120/10415]
loss: 0.502307  [ 7680/10415]
loss: 0.919433  [10240/10415]
Test Error: 
 Accuracy: 86.2%, Avg loss: 0.528720 

Epoch 2
-------------------------------
loss: 0.942880  [    0/10415]
loss: 0.271870  [ 2560/10415]
loss: 0.388227  [ 5120/10415]
loss: 0.295006  [ 7680/10415]
loss: 0.444954  [10240/10415]
Test Error: 
 Accuracy: 71.8%, Avg loss: 0.740645 

Epoch 3
-------------------------------
loss: 0.272083  [    0/10415]
loss: 0.288768  [ 2560/10415]
loss: 0.465035  [ 5120/10415]
loss: 0.194897  [ 7680/10415]
loss: 0.524283  [10240/10415]
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.339464 

Epoch 4
-------------------------------
loss: 0.457470  [    0/10415]
loss: 0.703181  [ 2560/10415]
loss: 0.366486  [ 5120/10415]
loss: 0.314253  [ 7680/10415]
loss: 0.233102  [10240/10415]
Test Error: 
 Accuracy: 60.4%, Avg loss: 1.469990 

Epoch 5
-----------------------

### Prediction visualization

### <font color=#FF0000>Visualization reference (non-used yet) <font>

#### single bounding box testing 

In [ ]:
def box(col, row, box_x, box_y):    # position + length / width (visualization usage)
    view = torch.zeros([28*3, 28*4], dtype=torch.float32)
    # set the box, outside the target with one pixel
    view[row-box_y:row+1, col-box_x] = 1
    view[row-box_y:row+1, col] = 1
    view[row-box_y, col-box_x:col+1] = 1
    view[row, col-box_x:col+1] = 1
    return view

In [ ]:
import torch
import matplotlib.pyplot as plt
from matplotlib.pyplot import pause
from random import randint
from copy import deepcopy

rand_x = randint(14, 27)
rand_y = randint(27, 28*3-1)
print("new start position [x, y] = [", rand_x,",", rand_y, "]")

rand_start = deepcopy(aug_sample)
box_origin = box(27, 21, 27, 15)    # array for visualization, the only manual input part
box_sample = deepcopy(box_origin)

plt.imshow(rand_start, cmap='gray')
plt.imshow(box_origin, cmap='gray', alpha=0.3)
pause(0.1)

# initial movement
for i in range(28):
    for j in range(28):
        rand_start[rand_y-i][rand_x-j] = rand_start[27-i][27-j]

for i in range(29):
    for j in range(29):
        box_origin[rand_y-i+1][rand_x-j+1] = box_origin[28-i][28-j]

# clean other part
rand_start[:rand_y-28+1, :] = 0
rand_start[:, rand_x+1:rand_x+28+1] = 0
box_origin[:rand_y-28, :] = 0       # 1-pixel cleaning region difference
box_origin[:, rand_x+1:rand_x+28+1] = 0

plt.imshow(rand_start, cmap='gray')
plt.imshow(box_origin, cmap='gray', alpha=0.3)
pause(0.1)


#### sequential bounding box visualization and random position

In [ ]:
from math import floor, ceil
from torch import cat
import matplotlib.pyplot as plt
from matplotlib.pyplot import pause
from copy import deepcopy

sequential box preparation

In [ ]:
# initialization
stride = 5
first_frame = deepcopy(box_sample)
next_frame = deepcopy(box_sample)
y_box = deepcopy(box_sample)

# show info
print(y_box.size())
plt.imshow(y_box, cmap='gray')
pause(0.1)

for i in range(floor((112-28)/stride)):
   # moving part (1-dim only)
   for j in range(30):  # bigger range to cover bounding box
      next_frame[:, 30+stride*(i+1)-(j+1)] = next_frame[:, 30+stride*i-(j+1)]
   next_frame[:, :stride*(i+1)] = 0  # clean other area
   
   # sequencing part
   if i == 0:
      y_box = stack((first_frame, next_frame)) 
   else:
      y_box = cat((y_box, next_frame.reshape(1, 28*3, 28*4)), dim=0)
      
   # # show info
   # print(y_box.size())
   # plt.imshow(y_box[i+1], cmap='gray')
   # pause(0.1)

parallel moving and concat last frames

In [ ]:
# sequential data preparation
y_box_moved = deepcopy(y_box)
y_moved = deepcopy(y)
rand_x = randint(14, 27)
rand_y = randint(27, 28*3-1)
print("new start position [x, y] = [", rand_x,",", rand_y, "]")

# # visualization for no parallel movement
# for i in range(len(y_box_moved)):
#     plt.imshow(y_moved[i], cmap='gray')
#     plt.imshow(y_box_moved[i], cmap='gray', alpha=0.3)
#     pause(0.1)

# all frames move left parellel
for i in range(floor((112-28)/stride)+1): 
    # moving items
    for j in range(28):
        for k in range(28):
            y_moved[i][rand_y-j][rand_x+stride*i-k] = y_moved[i][27-j][27+stride*i-k]
    # moving box
    for j in range(29):
        for k in range(29):
            y_box_moved[i][rand_y-j+1][rand_x+stride*i-k+1] = y_box_moved[i][28-j][28+stride*i-k]
          
    # clean other area
    y_moved[i][:rand_y-28+1, :] = 0
    y_moved[i][:, rand_x+stride*i+1:] = 0
    y_box_moved[i][:rand_y-28+1, :] = 0
    y_box_moved[i][:, rand_x+stride*i+1:] = 0
   
    # show info
    plt.imshow(y_moved[i], cmap='gray')
    plt.imshow(y_box_moved[i], cmap='gray', alpha=0.3)
    pause(0.1)

# initialization
frame_old = len(y_moved)   # number of old frames
next_frame = deepcopy(y_moved[frame_old-1])
next_frame_box = deepcopy(y_box_moved[frame_old-1])

# last frame moves right and cat
for i in range(ceil((28-rand_x)/stride)+1):
    # move next frame
    for j in range(28):
        if rand_x+stride*(frame_old+i)-j < 112:
           next_frame[:, rand_x+stride*(frame_old+i)-j] = next_frame[:, rand_x+stride*(frame_old+i-1)-j]
    for j in range(29):
        if rand_x+stride*(frame_old+i)-j+1 < 112:
           next_frame_box[:, rand_x+stride*(frame_old+i)-j+1] = next_frame_box[:, rand_x+stride*(frame_old+i-1)-j+1]
    # clean and cat
    next_frame[:, :rand_x+stride*(frame_old+i)-28+1] = 0
    y_moved = cat((y_moved, next_frame.reshape(1, 28*3, 28*4)), dim=0)
    next_frame_box[:, :rand_x+stride*(frame_old+i)-28+1] = 0
    y_box_moved = cat((y_box_moved, next_frame_box.reshape(1, 28*3, 28*4)), dim=0)
   
    # show info
    plt.imshow(y_moved[frame_old+i], cmap='gray')
    plt.imshow(y_box_moved[frame_old+i], cmap='gray', alpha=0.3)
    pause(0.1)

print(y_moved.size())